In [1]:
import sys
sys.path.append("d:\\OneDrive - Universiteit Utrecht\\Documents\\000 - Documenti\\PROJECTS\\x-tree-search")

In [2]:
from games.tic_tac_toe import TicTacToe
from games.game import GameAgent, User
from algorithms.minmax import MinMax
from explainers.minmax_explainer import MinMaxExplainer

# Define game

In [3]:
game = TicTacToe()

# Define tic-tac-toe scoring function for the minmax

In [4]:
import numpy as np
def scoring_function(node):
    """ Evaluate the Tic Tac Toe board state for the 'X' player's perspective """
    state = node.state
    score = 0
    
    # Possible lines to check (3 rows, 3 columns, 2 diagonals)
    lines = []
    # Rows and columns
    for i in range(3):
        lines.append(state[i, :])  # Row
        lines.append(state[:, i])  # Column
    # Diagonals
    lines.append(np.array([state[i, i] for i in range(3)]))  # Main diagonal
    lines.append(np.array([state[i, 2 - i] for i in range(3)]))  # Anti-diagonal

    for line in lines:
        if np.all(line == "X"):
            score += 100  # 'X' wins
        elif np.all(line == "O"):
            score -= 100  # 'O' wins
        elif np.count_nonzero(line == "X") == 2 and np.count_nonzero(line == "free") == 1:
            score += 10  # 'X' is one move away from winning
        elif np.count_nonzero(line == "O") == 2 and np.count_nonzero(line == "free") == 1:
            score -= 10  # 'O' is one move away from winning

    return score

# Define explainer

In [5]:
explainer = MinMaxExplainer()

# Play and Explain

In [6]:
opponent = GameAgent(MinMax(scoring_function), agent_id=0)
user = User(agent_id=1, ask_what = False)

In [7]:
game.set_players([opponent, user])

In [8]:
await game.start_on_jupyter()

[['O' 'X' ' ']
 ['O' ' ' ' ']
 ['X' ' ' ' ']]

Game paused.

[['O' 'X' ' ']
 ['O' ' ' ' ']
 ['X' ' ' ' ']]


In [9]:
default_settings = {
    'with_framework': 'lowlevel',
    'explanation_depth': 3 ,
    'print_implicit_assumptions': True,
    'assumptions_verbosity': 'verbose',
    'print_mode': 'logic'
}
explainer.configure_settings(default_settings)

In [10]:
choice = opponent.core.last_choice
explainer.explain_adjective(choice, 'best', explanation_depth=4)

0_3 is best ←
 	(assumption) By definition a node is "best" if it's "better" than all "siblings"
	∧ 0_3 has siblings = 0_0, 0_1, 0_2, 0_4, 0_5 ←
	 		(assumption) Definition of "siblings" is node.parent.children excluding node
	∧ 0_3 is better than 0_0 ←
	 		(assumption) By definition, node1 is "better" than node2 if node1 score >= node2 score
			∧ 0_3 has score = 0 ←
			 			0_3 is ¬(leaf) ←
						 				(assumption) Definition of "leaf" is node.is_leaf
						∧ (assumption) Internal nodes have scores from children
						∧ 0_3 has backpropagating child = 0_3_0 ←
						 				0_3 is opponent player turn
										∧ (assumption) We assume the opponent will do their best move.
										∧ 0_3_0 is worst
						∧ 0_3_0 has score = 0 ←
						 				0_3_0 is ¬(leaf)
										∧ (assumption) Internal nodes have scores from children
										∧ 0_3_0 has backpropagating child = 0_3_0_0 ∧ 0_3_0_0 has score = 0
			∧ 0_0 has score = -100 ←
			 			0_0 is ¬(leaf) ←
						 				(assumption) Definition of "lea

In [11]:
high_abstraction_settings = {
            'with_framework': 'highlevel',
            'explanation_depth': 4 ,
            'print_implicit_assumptions': False,
            'assumptions_verbosity': 'if_asked',
            'print_mode': 'verbal'
        }

explainer.configure_settings(high_abstraction_settings)

In [12]:
explainer.explain_adjective(choice, 'the best', explanation_depth=4)

0_3 is the best (because
 	0_3 has as possible alternative moves 0_0, 0_1, 0_2 (only showing relevant 3)
	and 0_3 is better than 0_0 (because
	 	it leads to a better position (because
	 		0_3 is not final move
			and 0_3 has as next possible move 0_3_0 (because
			 				0_3 is opponent player turn and 0_3_0 is the best the opponent can do)
			and 0_0 is not final move
			and 0_0 has as next possible move 0_0_2 (because
			 				0_0 is opponent player turn and 0_0_2 is the best the opponent can do)))
	and 0_3 is better than 0_1 (because
	 	it leads to a better position (because
	 		0_3 is not final move
			and 0_3 has as next possible move 0_3_0 (because
			 				0_3 is opponent player turn and 0_3_0 is the best the opponent can do)
			and 0_1 is not final move
			and 0_1 has as next possible move 0_1_2 (because
			 				0_1 is opponent player turn and 0_1_2 is the best the opponent can do)))
	and 0_3 is better than 0_2 (because
	 	it leads to a better position (because
	 		0_3 is not final

In [13]:
explainer.del_explanation_tactic('SubstituteQuantitativeExplanations', to_framework='highlevel')
#explainer.del_explanation_tactic('SkipConditionStatement', to_framework='highlevel')
explainer.del_explanation_tactic('OnlyRelevantComparisons', to_framework='highlevel', to_adjective='the best')

In [14]:
explainer.explain_adjective(choice, 'the best', explanation_depth=2)

0_3 is the best (because
 	0_3 has as possible alternative moves 0_0, 0_1, 0_2, 0_4, 0_5
	and 0_3 is better than 0_0 (because
	 		0_3 has as score 0 and 0_0 has as score -100)
	and 0_3 is better than 0_1 (because
	 		0_3 has as score 0 and 0_1 has as score -100)
	and 0_3 is better than 0_2 (because
	 		0_3 has as score 0 and 0_2 has as score -100)
	and 0_3 is better than 0_4 (because
	 		0_3 has as score 0 and 0_4 has as score -100)
	and 0_3 is better than 0_5 (because
	 		0_3 has as score 0 and 0_5 has as score -100))
